In [296]:
#!/usr/bin/env python3
#import your libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import LabelEncoder

In [334]:
train_transaction = pd.read_csv('/Users/krahman/work/fraud_detection/data/train_transaction.csv')
train_identity = pd.read_csv('/Users/krahman/work/fraud_detection/data/train_identity.csv')
# merging dataframes 
df_train = train_transaction.merge(train_identity, on='TransactionID', how='left')

In [322]:
# df_train

In [323]:
# train_transaction.head()

In [324]:
# train_identity.head()

In [325]:
# train_transaction.info();

In [326]:
# train_identity.info();

In [327]:
# df_train.shape

In [328]:
# train_transaction_sum = train_transaction.duplicated().sum()
# train_identity_sum = train_identity.duplicated().sum()
# columns = train_transaction.columns
# print('Train transaction duplicates: {}\nTrain identity duplicates: {} \n'.format(train_transaction_sum, train_identity_sum))
# # print('Train feature columns:\n', list(columns))

### Assigning columns to specific lists (cat, num, date, etc.)

In [335]:
# dropping columns with more than 50% missing data
length_df = df_train.shape[0]/2
list_temp = []
for val in df_train.columns:
    if np.sum(df_train[val].isnull()) > length_df:
        list_temp.append(val)   
df_train = df_train.drop(list_temp, axis=1)

In [336]:
class Preprocessing:
    def __init__(self):
        '''initialize variables and column names for null dataframe'''
        self.df_train = df_train
        self.list_col = []
        self.list_total = []
        self.dict_unique = {}
        self.list_datatype = []
        self.list_unique_val = []
        self.list_mode_count = []
        self.list_mode_value = []
        self.list_mode_count_perc = []
        self.list_unique_total = []
        self.list_unique_first_10 = []
        self.column_names = ['col_name', 'total_null', 'datatype', 'total_unique',
                             'mode_value', 'mode_count', 'mode_percentage']

    def missing_values(self):
        '''check for null values and add to null dataframe if more than 0 nulls exist'''
        for val in df_train.columns:
            total_null = np.sum(df_train[val].isnull())
            if total_null > 0:
                self.list_col.append(val)
                self.list_total.append(total_null)
                self.list_datatype.append(df_train[val].dtype)
                self.list_unique_total.append(len(df_train[val].unique()))
                self.list_unique_val.append(df_train[val].unique())
                self.list_mode_value.append(df_train[val].mode()[0])
                val_counts = max(df_train[val].value_counts())
                self.list_mode_count.append(val_counts)
                self.list_mode_count_perc.append(val_counts/len(df_train))
                val_unique = df_train[val].unique()
                self._create_dict(val_unique, df_train, val)
        df_null_info = self._create_dataframe()
        df_null_info = self._create_df_unique(df_null_info)
        self._summary(df_null_info)
        self._fillna(df_null_info)
        return df_null_info
    
    def _create_dict(self, val_unique, df_train, val):
        '''create dictionary of unique values for each column'''
        if (len(val_unique) > 99) and isinstance(df_train[val], object):  
            self.dict_unique.update([(val,0)])
        if (len(val_unique) > 99) and not isinstance(df_train[val], object):
            self.dict_unique.update([(val,0)])
        if len(val_unique) < 100:
            self.dict_unique.update([(val, val_unique)])

    def _create_dataframe(self):
        '''create main dataframe'''
        df_null_info = pd.DataFrame()
        counter = -1
        for list_val in [self.list_col, self.list_total, self.list_datatype, self.list_unique_total,
                        self.list_mode_value, self.list_mode_count, self.list_mode_count_perc]:
            counter = counter + 1
            col_title = self.column_names[counter]
            df = pd.DataFrame(list_val, columns=[col_title])
            df_null_info = pd.concat([df_null_info, df], axis=1)
        return df_null_info
    
    def _summary(self, df_null_info):
        val = df_null_info.shape[0]
        print('There were ' + str(val) + ' columns with null values.')
    
    def _create_df_unique(self, df_null_info):
        '''create unique values dataframe'''
        series_unique = pd.Series(self.dict_unique)
        df_unique = pd.DataFrame(series_unique).reset_index()
        df_unique = df_unique.rename(columns={'index':'col_name', 0:'unique'})
        df_null_info = df_null_info.merge(df_unique, how='left', 
                                          left_on='col_name', right_on='col_name')
        df_null_info.to_csv('/Users/krahman/work/fraud_detection/saved_files/df_null_info.csv')
        return df_null_info
    
    def _fillna(self, df_null_info):
        '''fill null values of df_train with mode'''
        total_null_columns = sum(np.sum(self.df_train.isnull()))
        if total_null_columns > 0:
            for val in df_null_info.col_name:
                val_mode = self.df_train[val].mode()[0]
                self.df_train[val] = self.df_train[val].fillna(val_mode)
                
    def impute_features(self):
        df_temp = pp.df_train
        for val in col_cat:
            if len(df_temp[val].unique()) < 10:
                print('dummies encoded: ' + str(val))
                df_dumm = pd.get_dummies(df_temp[val], prefix=val)
                df_temp = pd.concat([df_temp, df_dumm], axis=1)
            else:
                le = LabelEncoder()
                df_temp[val] = le.fit_transform(df_temp[val])
                print('label encoded: ' + str(val))
        print('new dataframe shape:' + str(df_temp.shape))
        return df_temp

pp = Preprocessing()
df_null_info = pp.missing_values()
# df_null_info

# determine what to do with columns that have too many unique values... obviously.. types of solutions
# would be to put "MISSING" for those that dont have an email address... but you will need to evaluate 
# and make instead a counter of unique values, then append that and look at the CSV via google sheets. use 
# something like the code below 

# Planning - our preprocessing method must automatically drop missing values, but we can't do that because
# we need to see about filling them in first, then decide if we need to drop them. Right now, we need to
# create a dataframe that shows unique values for each column with missing values. 

# we need to look at each variable and see if it's unique or categorical. We need to use possibly PCA...? How do
# we handle so many variables? 

There were 200 columns with null values.


In [337]:
pp.df_train = pp.impute_features()

label encoded: addr1
label encoded: addr2
dummies encoded: ProductCD
label encoded: P_emaildomain
label encoded: card1
label encoded: card2
label encoded: card3
dummies encoded: card4
label encoded: card5
dummies encoded: card6
dummies encoded: M1
dummies encoded: M2
dummies encoded: M3
dummies encoded: M4
dummies encoded: M6
new dataframe shape:(590540, 244)


In [339]:
pp.df_train.dtypes.head(50)
# NEXT, dummies encoding is not encoding or saving properly, fix this before moving onto PCA. 

TransactionID       int64
isFraud             int64
TransactionDT       int64
TransactionAmt    float64
ProductCD          object
card1               int64
card2               int64
card3               int64
card4              object
card5               int64
card6              object
addr1               int64
addr2               int64
P_emaildomain       int64
C1                float64
C2                float64
C3                float64
C4                float64
C5                float64
C6                float64
C7                float64
C8                float64
C9                float64
C10               float64
C11               float64
C12               float64
C13               float64
C14               float64
D1                float64
D2                float64
D3                float64
D4                float64
D10               float64
D11               float64
D15               float64
M1                 object
M2                 object
M3                 object
M4          

In [247]:
# Once we decide on label encode, or impute, we will need to eventually use PCA. 
# how do we approach this? We have categorical columns with lots of numbers.. it doesnt make sense to 
# label encode at hte same time we risk having high cardinality... which means we must just label 
# encoding anything below 10 for now, then switch to 59, see how that performs, apply, PCA, 
# test again.. run the model.. So here are our steps 

# NEXT, 1. dummies encode anything less than 10, 2. label encode the rest 3. apply PCA 3. run the model 

In [189]:
# c is num, ex, how many addresses associated with card
col_c = [c for c in df_train.columns if c.startswith('C') and (len(c)==2 or len(c)==3)]
# d is num, time/days between transactions
col_d = [d for d in df_train.columns if d.startswith('D') and (len(d)==2 or len(d)==3)]
# m is date of transaction
col_m = [m for m in df_train.columns if m.startswith('M') and (len(m)==2 or len(m)==3)]
# v is num, features created by vesta such as ranking, counting. entity relationships, etc. 
col_v = [v for v in df_train.columns if v.startswith('V') and (len(v)==2 or len(v)==3 or len(v)==4)]
# i is identity information like network and digital signature associated with transaction
col_i = [i for i in df_train.columns if i.startswith('id_') and len(i)==5]
# ca is cat, card information such as card type, etc. 
col_card = [ca for ca in df_train.columns if ca.startswith('card')]

In [223]:
# D = time elapsed between each transaction, card = card information, C = counting, ie how many addresses 
# associated with card, M=True/False, V created features on ranking, counting, etc. 

# column id and target
col_id = ['TransactionID']
col_target = ['isFraud']

# converting categorical columns with numerical values to string types.
col_cat_to_obj = ['addr1','addr2','card1','card2', 'card3', 'card5']
for val in col_cat_to_obj:
    df_train[val] = df_train[val].astype(str)

# categorical columns
col_cat = ['addr1','addr2','ProductCD',"P_emaildomain"] + col_card + col_m

# C counter, D is time elapsed between transactions, V feature engineered variables by firm
col_num = ['TransactionAmt'] + col_c + col_d + col_v

# figure out how to handle this. What do these dates mean? Do certain dates have more fraud occurences?
col_date = ['TransactionDT'] 

# boolean columns. convert via dummy variable. We dont know if true/false is better than one or the other. 
# col_bool = col_m

# confirming all columns are accounted for
print('Total columns: ' + str(len(col_cat + col_num + col_date + col_id + col_i + col_target)))

# col_all = col_cat + col_num + col_date + col_bool + col_id + col_target
# columns removed dist1, dist2, R_emaildomain, DeviceInfo, DeviceType 

Total columns: 220


In [205]:
pp.df_train.head(5)

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321
0,2987000,0,86400,68.5,W,13926,321.0,150.0,discover,142.0,...,0.0,0.0,0.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,...,135.0,0.0,0.0,0.0,50.0,1404.0,790.0,0.0,0.0,0.0
4,2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
# We need to clean the data before thinking about applying PCA. 


# 1. impute all objects columns with one hot encoding
# 1.1 What's a categorical? 
# We know the V's are ranking.. we need to discern meaning, 
# 

# 2. figure out if we should do pca next. we should do that next..
# 3. use decision trees to determine important features. We don't need to do label encoding for this. If 
#    our training model is too slow... we need to implement PCA before further EDA via decision trees. 
# 4. Once we finish decision trees, we can continue EDA. 
# 5. If dt is too slow, we will need to impute our data for PCA. Let's impute now.

In [147]:
for val in pp.df_train.columns:
    if (pp.df_train[pp.df_train[val]=='nan'].shape[0]) > 0:
        print(val)

In [34]:
# pp.df_train
# lets find out which columns are object... 
list_col_object = []
for val in pp.df_train.columns:
    if pp.df_train[val].dtype=='O':
        list_col_object.append(val)
        
pp.df_train[list_col_object]
# for card2, nan was the most commonly seen value... so it imputed that...? 

,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,P_emaildomain,M1,M2,M3,M4,M6
0,W,13926,nan,150.0,discover,142.0,credit,315.0,87.0,gmail.com,T,T,T,M2,T
1,W,2755,404.0,150.0,mastercard,102.0,credit,325.0,87.0,gmail.com,T,T,T,M0,T
2,W,4663,490.0,150.0,visa,166.0,debit,330.0,87.0,outlook.com,T,T,T,M0,F
3,W,18132,567.0,150.0,mastercard,117.0,debit,476.0,87.0,yahoo.com,T,T,T,M0,F
4,H,4497,514.0,150.0,mastercard,102.0,credit,420.0,87.0,gmail.com,T,T,T,M0,F
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590535,W,6550,nan,150.0,visa,226.0,debit,272.0,87.0,gmail.com,T,T,T,M0,F
590536,W,10444,225.0,150.0,mastercard,224.0,debit,204.0,87.0,gmail.com,T,F,F,M0,T
590537,W,12037,595.0,150.0,mastercard,224.0,debit,231.0,87.0,gmail.com,T,F,F,M0,T
590538,W,7826,481.0,150.0,mastercard,224.0,debit,387.0,87.0,aol.com,T,T,T,M0,T


In [ ]:
# for val in pp.df_train.columns:
#     if pp.df_train[pp.df_train[val]=='nan']

In [ ]:
list_col_object = []
for val in pp.df_train.columns:
    if pp.df_train[val]:
        list_col_object.append(val)

In [38]:
np.sum(pp.df_train[list_col_object].isnull())
pp.df_train['card2'].unique()
pp.df_train[df_train['card2']=='nan']

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321
0,2987000,0,86400,68.50,W,13926,nan,150.0,discover,142.0,...,0.0,0.000000,0.000000,0.000000,0.0,117.0,0.0,0.0,0.0,0.0
123,2987123,0,88549,59.00,W,6550,nan,150.0,visa,226.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
135,2987135,0,88671,3000.00,W,2616,nan,150.0,discover,102.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
213,2987213,0,89867,499.95,W,6957,nan,150.0,visa,166.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
223,2987223,0,89948,54.00,W,2616,nan,150.0,discover,102.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
589755,3576755,0,15794787,661.92,W,10476,nan,143.0,visa,226.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
589889,3576889,0,15797613,39.00,W,12219,nan,150.0,visa,166.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
590005,3577005,0,15799999,40.00,S,14823,nan,150.0,visa,166.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
590068,3577068,0,15801472,34.50,W,8187,nan,150.0,visa,166.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
sum(np.sum(pp.df_train.isnull()))

0

In [ ]:
# We need to clean the data before thinking about applying PCA. 
# 1. Determine which columns are continuous, which are ranking.
# 2. Determine which columns are bool (easy)
# 3. Determine which columns are categorical, then impute with pandas (we dont know which columns means what
#    so we cant assume True or better than False, etc.)
# 4. After 

# 1. impute all objects columns with one hot encoding
# 1.1 What's a categorical? 
# We know the V's are ranking.. we need to discern meaning, 

# 2. figure out if we should do pca next. we should do that next..
# 3. then stand up the model. 


In [17]:
# plt.bar(df_train['V14'])
# sns.barplot(df_train['V196'])
# we need to imput the mode here.. 
# df_train['V14'].mode()
# df_train['V22'].unique()
# for val in col_v:
#     print(val)
#     print(df_train[val].unique())
# we ned to descern what is a 0 1 outcome then impute.

# col = 'V290'
# series_temp = df_train[col].fillna(df_train[col].mode()[0])
# plt.hist(series_temp);
# missing_val = np.sum(df_train[col].isnull())
# print('Missing values: ' + str(missing_val))
# print("REAL VALUE COUNTS: ")
# df_train[col].value_counts().head()

# col = 'card4'
# series_temp = df_train[col].fillna(df_train[col].mode()[0])
# plt.hist(series_temp);
# df_train[col].value_counts()

# col = 'D1'
# series_temp = df_train[col].fillna(df_train[col].mean())
# plt.hist(series_temp);
# df_train['D1'].value_counts()